# 패키지 임포트

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import re
import json
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

# 데이터 로드

In [2]:
train_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-train-stemmed.csv')
valid_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-dev-stemmed.csv')
test_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-test-stemmed.csv')
train_df.head()

,text,edge,intent
0,champion product approv stock split champion p...,champion product approv stock split champion p...,earn
1,comput termin system cpml complet sale comput ...,comput termin system cpml complet sale comput ...,acq
2,cobanco inc cbco year net shr ct dlr net asset...,cobanco inc cbco year net shr ct dlr net asset...,earn
3,intern inc qtr jan oper shr loss two ct profit...,intern inc qtr jan oper shr loss two ct profit...,earn
4,brown forman inc bfd qtr net shr dlr ct net ml...,brown forman inc bfd qtr net shr dlr ct net ml...,earn


In [3]:
# train_df와 valid_df 결합
train_val_df = pd.concat([train_df, valid_df])

In [4]:
textcuboid=np.load('C:/Users/User/Desktop/r8/1-Channel textcuboid_r8(bert).npy')
textcuboid_test=np.load('C:/Users/User/Desktop/r8/1-Channel textcuboid_test_r8(bert).npy')

# 전처리

In [5]:
y_train=list(train_val_df['intent'])
y_test=list(test_df['intent'])

In [6]:
y=y_train+y_test

In [7]:
encoder=LabelEncoder()
encoder.fit(y)
label=encoder.transform(y)

y_train=list(label[:5484])
y_test=list(label[5484:])

In [8]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [9]:
textcuboid_train=textcuboid[:4937]
textcuboid_val=textcuboid[4937:]
y_train1=y_train[:4937]
y_val=y_train[4937:]

In [10]:
y_test1=y_test

# 모델 빌드


In [11]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, save_freq=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [12]:
#ELMo channel의 학습
BERT_input = Input(shape=(169, 768))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(BERT_input)
pooling = GlobalMaxPooling1D()(conv1)
pooling = Dropout(0.5)(pooling)
BERT_x = Dense(256, activation='relu')(pooling)
BERT_x = Dropout(0.5)(BERT_x)
output_layer = Dense(8, activation='softmax')(BERT_x)

model = Model(inputs=BERT_input, outputs=output_layer)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 169, 768)]        0         
                                                                 
 conv1d (Conv1D)             (None, 169, 1024)         787456    
                                                                 
 global_max_pooling1d (Globa  (None, 1024)             0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                             

# 모델 학습

In [20]:
#모델 학습
history = model.fit(textcuboid_train, y_train1,callbacks=callbacks, epochs=50,batch_size=256,validation_data=(textcuboid_val, y_val))

Epoch 1/50
20/20 [==============================] - 1s 55ms/step - loss: 4.1829 - accuracy: 0.5305 - val_loss: 1.3521 - val_accuracy: 0.5996 - lr: 0.0010
Epoch 2/50
20/20 [==============================] - 1s 35ms/step - loss: 0.9618 - accuracy: 0.6836 - val_loss: 0.9642 - val_accuracy: 0.6563 - lr: 0.0010
Epoch 3/50
20/20 [==============================] - 1s 39ms/step - loss: 0.7625 - accuracy: 0.7460 - val_loss: 0.7746 - val_accuracy: 0.7203 - lr: 0.0010
Epoch 4/50
20/20 [==============================] - 1s 38ms/step - loss: 0.6570 - accuracy: 0.7733 - val_loss: 0.7352 - val_accuracy: 0.7367 - lr: 0.0010
Epoch 5/50
20/20 [==============================] - 1s 37ms/step - loss: 0.5602 - accuracy: 0.8041 - val_loss: 0.5920 - val_accuracy: 0.7934 - lr: 0.0010
Epoch 6/50
20/20 [==============================] - 1s 35ms/step - loss: 0.4737 - accuracy: 0.8388 - val_loss: 0.4745 - val_accuracy: 0.8592 - lr: 0.0010
Epoch 7/50
20/20 [==============================] - 1s 38ms/step - loss: 0.4

In [21]:
model.evaluate(textcuboid_test,y_test1)

69/69 [==============================] - 0s 3ms/step - loss: 0.1364 - accuracy: 0.9667


[0.13640496134757996, 0.966651439666748]

In [19]:
model = load_model('./save model/best_model_1-Channel Tensor(BERT).h5')
model.evaluate(textcuboid_test,y_test1)

69/69 [==============================] - 0s 4ms/step - loss: 0.1333 - accuracy: 0.9676


[0.1332753747701645, 0.96756511926651]

In [20]:
y_pred = model.predict(textcuboid_test)
y_pred_labels = np.argmax(y_pred, axis=1)
macro_f1 = f1_score(y_test, y_pred_labels, average='macro')
print("Macro_F1 스코어:", macro_f1)
weighted_f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("weighted_F1 스코어:", weighted_f1)

69/69 [==============================] - 0s 3ms/step
Macro_F1 스코어: 0.9284601445543716
weighted_F1 스코어: 0.9670420918302938


In [ ]:
45에포크 drop1,2 활성화 1024 96.8